## Mapbox choropleth that works with Plotly 3.0.0

In [118]:
import numpy as np
import pandas as pd
from ast import literal_eval
import topojson
import json
mapbox_access_token = 'pk.eyJ1IjoibWF0dGxldml0YW4iLCJhIjoiY2pzNWcydjVyMGZkazRhb2JucGV1MjMydCJ9.lfu1STda8KYLGVnCyX6y8Q'

Data files for this notebook are available here:

[https://github.com/empet/Plotly-plots/blob/master/Data/tx_counties.topojson](https://github.com/empet/Plotly-plots/blob/master/Data/tx_counties.topojson)

[https://github.com/empet/Plotly-plots/blob/master/Data/tx_unemployment.csv](https://github.com/empet/Plotly-plots/blob/master/Data/tx_unemployment.csv)    

Read  the topojson file for the  [Texas counties](https://github.com/TNRIS/tx.geojson/tree/master/counties), 
and convert it to a geojson dict: 

In [142]:
topoJSON=pd.read_json(path_or_buf=
                      '/Users/mattlevitan/Downloads/cbsa.tiger2013.json',
                      typ = 'FeatureCollection')

In [148]:
topoJSON

type                                        FeatureCollection
features    [{'type': 'Feature', 'properties': {'CSAFP': '...
dtype: object

In [22]:
import dash_core_components as dcc

dcc.Dropdown(
    options=[
        {'label': 'New York City', 'value': 'NYC'},
        {'label': 'Montréal', 'value': 'MTL'},
        {'label': 'San Francisco', 'value': 'SF'}
    ],
    value='MTL'
)


Dropdown(options=[{'label': 'New York City', 'value': 'NYC'}, {'label': 'Montréal', 'value': 'MTL'}, {'label': 'San Francisco', 'value': 'SF'}], value='MTL')

In [155]:
topoJSON['features'][0]['properties'].keys()

dict_keys(['CSAFP', 'CBSAFP', 'GEOID', 'NAME', 'NAMELSAD', 'LSAD', 'MEMI', 'MTFCC', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON'])

In [93]:
# topo_features = topoJSON['objects']['tx_counties']['geometries']
# scale = topoJSON['transform']['scale']
# translation = topoJSON['transform']['translate']

In [20]:
my_dict= {'1':'a','2':'b'
my_dict

TypeError: unhashable type: 'dict'

Define the coresponding geojson dict:

In [94]:
# geoJSON=dict(type= 'FeatureCollection', 
#              features = [])

# for k, tfeature in enumerate(topo_features):
#     geo_feature = dict(id=k, type= "Feature")
#     geo_feature['properties'] = tfeature['properties']
#     geo_feature['id']=tfeature['id']
#     geo_feature['geometry'] = topojson.geometry(tfeature, topoJSON['arcs'], scale, translation)    
#     geoJSON['features'].append(geo_feature)  

In [ ]:
# var somedata = {
#    "Pizza" : ["Cheese", "Bread", "Sauce"],
#    "Spaghetti" : ["Pasta", "Sauce", "Meatballs"]
# }

# Object.keys(somedata)       // returns  ["Pizza", "Spaghetti"]
# Object.keys(somedata)[0]   // returns  "Pizza"
# Object.keys(somedata)[1]   // returns "Spaghetti"

# somedata["Pizza"]    // returns ["Cheese", "Bread", "Sauce"]
# somedata["Spaghetti"]   // returns  ["Pasta", "Sauce", "Meatballs"]

# //---------------
# Object.values(somedata)        // returns  [Array(3), Array(3)]
# Object.values(somedata)[0]    // returns ["Cheese", "Bread", "Sauce"]
# Object.values(somedata)[1]    // returns ["Pasta", "Sauce", "Meatballs"]

Get the lon and lat of a central location for each county:

In [98]:
# lons=[]
# lats=[]
# for k in range(len(geoJSON['features'])):
#     county_coords=np.array(geoJSON['features'][k]['geometry']['coordinates'][0])
#     m, M =county_coords[:,0].min(), county_coords[:,0].max()
#     lons.append(0.5*(m+M))
#     m, M =county_coords[:,1].min(), county_coords[:,1].max()
#     lats.append(0.5*(m+M))

In [99]:
counties=[geoJSON['features'][k]['properties']['id'] for k in range(len(geoJSON['features']))]
tx_ids=[geoJSON['features'][k]['id'] for k in range(len(geoJSON['features']))]      

Read the unemployment file, extracted for the Texas state from [that](https://gist.github.com/mbostock/4060606#file-unemployment-tsv) corresponding to whole US.

In [166]:


df=pd.read_clipboard()
   

In [167]:

df.head()

,msa,prps_refi
0,10180,2382
1,10380,1002
2,10420,14260
3,10500,2670
4,10540,3397


In [168]:
df.index = df['msa']

In [1]:
rate=[df.loc[id, 'rate'] for id in txids]
zmin=min(rate)
zmax=max(rate)

NameError: name 'cbsa_ids' is not defined

In [103]:
sources=[]
for feat in geoJSON['features']: 
        sources.append({"type": "FeatureCollection", 'features': [feat]})

Define a function that maps a value in the range [vmin, vmax] to the corresponding  color in a given colorscale:

In [104]:
def get_color_for_val(val, vmin, vmax, pl_colorscale):
    if vmin >= vmax:
        raise ValueError('vmin should be < vmax')
        
    plotly_scale, plotly_colors = list(map(float, np.array(pl_colorscale)[:,0])), np.array(pl_colorscale)[:,1]  
    colors_01=np.array(list(map(literal_eval,[color[3:] for color in plotly_colors] )))/255.#color codes in [0,1]
    
    v= (val - vmin) / float((vmax - vmin)) #here val is mapped to v in[0,1]
    #find two consecutive values in plotly_scale such that   v belongs to the corresponding interval
    idx = 0
   
    while(v > plotly_scale[idx+1]): 
        idx+=1
    left_scale_val = plotly_scale[idx]
    right_scale_val = plotly_scale[idx+ 1]
    vv = (v - left_scale_val) / (right_scale_val - left_scale_val)##attn! this code works well if the plotly_scale is 
                                                              #sorted ascending, and there are no duplicates in
                                                              # plotly_scale
    #get the  [0,1]-valued color code representing the rgb color corresponding to val
    val_color01 = colors_01[idx]+vv*(colors_01[idx + 1]-colors_01[idx])
    val_color_0255 = list(map(np.uint8, 255*val_color01+0.5))
    return 'rgb'+str(tuple(val_color_0255))

The colorscale definition:

In [105]:
pl_colorscale= [[0.0, 'rgb(255, 255, 204)'],
                [0.35, 'rgb(161, 218, 180)'],
                [0.5, 'rgb(65, 182, 196)'], 
                [0.6, 'rgb(44, 127, 184)'],
                [0.7, 'rgb(8, 104, 172)'],
                [1.0, 'rgb(37, 52, 148)']] 

Compute the color corresponding to each county, according to its unemployment rate:

In [106]:
facecolor=[get_color_for_val(r, zmin, zmax, pl_colorscale)  for r in rate] 

In [107]:
text=[c+'<br>Unemployment rate: '+'{:0.2f}'.format(r)+'%' for c, r in zip(counties, rate)]

In [108]:
USA = dict(type='scattermapbox',
             lat=lats, 
             lon=lons,
             mode='markers',
             text=text,
             marker=dict(size=1, color=facecolor),
             showlegend=False,
             hoverinfo='text'
            )

In [109]:
layers=[dict(sourcetype = 'geojson',
             source =sources[k],
             below="water", 
             type = 'fill',   
             color = facecolor[k],
             opacity=0.8
            ) for k in range(len(sources))]

In [110]:
layout = dict(title='Mapbox Choropleth<br>Texas unemployment',
              font=dict(family='Balto'),
              autosize=False,
              width=800,
              height=800,
              hovermode='closest',
   
              mapbox=dict(accesstoken=mapbox_access_token,
                          layers=layers,
                          bearing=0,
                          center=dict(
                          lat=31.4638, 
                          lon=-99.98),
                          pitch=0,
                          zoom=5,
                    ) 
              )

fig = dict(data=[Texas], layout=layout)

In [114]:
import plotly.plotly as py
py.sign_in('levitan.matt','ezbwoqyVCTSUpcY0pcLc')
py.iplot(fig, filename='Texas-counties')

In [ ]:
import plotly.graph_objs as go
fw=go.FigureWidget(fig)
fw